In [1]:
import os
import cv2
import numpy as np
from imutils import paths

In [2]:
from sklearn.preprocessing import LabelBinarizer

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
datapath = r"C:\Users\Mahibub Sab\Desktop\6th sem\pro_6thsem\dataset"
outputmodel = r"C:\Users\Mahibub Sab\Desktop\6th sem\pro_6thsem\dataset\videoclassificationmodel"
outputlabelbinarizer = r"C:\Users\Mahibub Sab\Desktop\6th sem\pro_6thsem\dataset\videoclassificationmodel\videoclassificationbinarizer"
epoch= 25

In [5]:
from keras.preprocessing.image import ImageDataGenerator

In [6]:
Sports_Labels = set(['boxing', 'swimming', 'table_tennis', "basketball", "chess", "cricket"])
print("Images is being loaded.......")
pathToImages = list(paths.list_images(datapath))
data = []
labels = []


for images in pathToImages:
    label = images.split(os.path.sep)[-2]
    if label not in Sports_Labels:
        continue
    image = cv2.imread(images)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (244,244))
    data.append(image)
    labels.append(label)

Images is being loaded.......


In [7]:
data = np.array(data)
labels = np.array(labels)
#hot encoded values as 0,1,2
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

In [8]:
(X_train, X_test, Y_train, Y_test) =train_test_split(data, labels, test_size=0.25, stratify=labels, random_state=42)

In [9]:
trainingAugmentation = ImageDataGenerator(
    rotation_range=30,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"
)
validationAugmentation = ImageDataGenerator()
mean = np.array([123.68, 116.779, 103.999] , dtype="float32")
trainingAugmentation.mean = mean
validationAugmentation.mean = mean

In [10]:
from keras.layers import Input
from keras.layers.pooling import AveragePooling2D
from keras.applications import ResNet50
from keras.layers.core import Flatten
from keras.layers.core import Dense

from keras.layers.core import Dropout

from keras.models import Model

In [11]:
baseModel = ResNet50(weights="imagenet", include_top= False, input_tensor=Input(shape=(224,244,3)))

headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7,7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(512, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(len(lb.classes_), activation="softmax")(headModel)
model = Model(inputs=baseModel.input , outputs=headModel)

for basemodelLayers in baseModel.layers:
    basemodelLayers.trainable = False

In [12]:
from keras.optimizers import SGD

In [13]:
opt = SGD(lr=0.0001, momentum=0.9, decay=1e-4/epoch)

In [16]:
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

In [29]:
History = model.fit_generator( 
    trainingAugmentation.flow(X_train, Y_train, batch_size=32),
    steps_per_epoch=len(X_train) // 32,
    validation_data=validationAugmentation.flow(X_test, Y_test),
    validation_step=len(X_test) // 32,
    epochs=epoch
)

Instructions for updating:
Please use Model.fit, which supports generators.


TypeError: fit_generator() got an unexpected keyword argument 'validation_step'